<a href="https://colab.research.google.com/github/Lonely52Hz/ID2223_Scalable_Machine_Learning_and_Deep_Learning/blob/main/ID2223_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set environment in colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 66.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=fa1d8ff9bc2844c184870fb540cfafd6be1f665ff21d30246af19774da69de78
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port", "4050").getOrCreate()
from pyspark.sql.functions import *

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [5]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models, losses, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
import math
from datetime import datetime
import torch
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import sklearn
from scipy import stats

     |████████████████████████████████| 78 kB 4.8 MB/s 
     |████████████████████████████████| 3.3 MB 29.1 MB/s 
     |████████████████████████████████| 3.3 MB 48.8 MB/s 
     |████████████████████████████████| 1.2 MB 46.1 MB/s 
     |████████████████████████████████| 61 kB 625 kB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 596 kB 77.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=478df86d4db2be0970124a0ed11b7d02c20557a7fcd61fed30c520c0c6efcabc
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Regression

In [6]:
# read files
train = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
test = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-test.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
dev = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-dev.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')

train.show(5)
train.printSchema()

+-------------+------+--------+-----+-----+--------------------+--------------------+
|        genre|  file|    year|index|score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+-----+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|  5.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|  3.8|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|  3.8|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|  2.6|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9| 4.25|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+-----+--------------------+--------------------+
only showing top 5 rows

root
 |-- genre: string (nullable = true)
 |-- file: string (nullable = true)
 |-- year: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- sentenc

In [7]:
# map [0, 5] to [-1, 1]
train = train.withColumn('score', col('score')/2.5 - 1)
test = test.withColumn('score', col('score')/2.5 - 1)
dev = dev.withColumn('score', col('score')/2.5 - 1)

train.show(5)
train.describe(['score']).show()

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|                 0.7|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

+-------+-------------------+
|summary|              score|
+-----

In [8]:
# find rows with missing value
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()
dev.select([count(when(col(c).isNull(), c)).alias(c) for c in dev.columns]).show()

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        6|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+



In [9]:
# delete rows with missing value
train = train.na.drop()
test = test.na.drop()
dev = dev.na.drop()

train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
train.show(5)

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        0|
+-----+----+----+-----+-----+---------+---------+

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|     

In [10]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),\
                               pooling_mode_mean_tokens=True,\
                               pooling_mode_cls_token=False,\
                               pooling_mode_max_tokens=False)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [12]:
loss = losses.CosineSimilarityLoss(model=model)

In [13]:
# Transform the data so that it can be used to train and test
train_example = []
test_example = []
dev_example = []

df_train = train.toPandas()
df_test = test.toPandas()
df_dev = dev.toPandas()

for row in df_train.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  train_example.append(inp_example)

for row in df_test.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  test_example.append(inp_example)

for row in df_dev.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  dev_example.append(inp_example)

train_dataloader = DataLoader(train_example, shuffle=True, batch_size=16)

In [14]:
# train
'''
num_epochs = 1
model_save_path = 'output/training_stsbenchmark_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name='sts-dev')
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up
model.fit(train_objectives=[(train_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=model_save_path)
'''
# to save time, we use the previously trained model instead of training it everytime

'\nnum_epochs = 1\nmodel_save_path = \'output/training_stsbenchmark_\'+\'bert-base-uncased\'+\'-\'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n\nevaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name=\'sts-dev\')\nwarmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up\nmodel.fit(train_objectives=[(train_dataloader, loss)],\n          optimizer_class=torch.optim.Adam,\n          optimizer_params={\'lr\': 2e-5},\n          epochs=num_epochs,\n          warmup_steps=warmup_steps,\n          evaluator=evaluator,\n          evaluation_steps=1000,\n          output_path=model_save_path)\n'

In [38]:
# built-in evaluation

regression_model_path = '/content/drive/MyDrive/output/training_stsbenchmark_bert-base-uncased-2021-12-10_21-53-26'
model = SentenceTransformer(regression_model_path)
# model = SentenceTransformer(model_save_path)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_example, name='sts-test')
model.evaluate(test_evaluator)

0.5739843881319436

In [39]:
# evaluation using Spearmean Correlation, they have the same result
sentencea = model.encode(df_test['sentenceA'])
sentenceb = model.encode(df_test['sentenceB'])
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(sentencea, sentenceb)
print(cos_sim)
result = stats.spearmanr(cos_sim, df_test['score'])
print(result)

[0.6670458  0.9073092  0.6678997  ... 0.07020611 0.38499272 0.02931237]
SpearmanrResult(correlation=0.5739843881319436, pvalue=2.127869343158135e-121)


# Classification

In [17]:
# load data from snli
c_train = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_train.jsonl')
c_test = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_test.jsonl')
c_dev = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_dev.jsonl')
c_train.show(5)

+----------------+----------------+-------------+-------------------+--------------------+----------------------+--------------------+--------------------+----------------------+--------------------+
|annotator_labels|       captionID|   gold_label|             pairID|           sentence1|sentence1_binary_parse|     sentence1_parse|           sentence2|sentence2_binary_parse|     sentence2_parse|
+----------------+----------------+-------------+-------------------+--------------------+----------------------+--------------------+--------------------+----------------------+--------------------+
|       [neutral]|3416050480.jpg#4|      neutral|3416050480.jpg#4r1n|A person on a hor...|  ( ( ( A person ) ...|(ROOT (S (NP (NP ...|A person is train...|  ( ( A person ) ( ...|(ROOT (S (NP (DT ...|
| [contradiction]|3416050480.jpg#4|contradiction|3416050480.jpg#4r1c|A person on a hor...|  ( ( ( A person ) ...|(ROOT (S (NP (NP ...|A person is at a ...|  ( ( A person ) ( ...|(ROOT (S (NP (DT ...|


In [18]:
c_train=c_train.drop('_corrupt_record')
c_train=c_train.na.drop()
c_train=c_train.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_dev=c_dev.drop('_corrupt_record')
c_dev=c_dev.na.drop()
c_dev=c_dev.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_test=c_test.drop('_corrupt_record')
c_test=c_test.na.drop()
c_test=c_test.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_df_train=c_train.toPandas()
c_df_test=c_test.toPandas()
c_df_dev=c_dev.toPandas()
# get rid of '-'
c_df_train = c_df_train[~c_df_train['label'].isin(['-'])]
c_df_test = c_df_test[~c_df_test['label'].isin(['-'])]
c_df_dev = c_df_dev[~c_df_dev['label'].isin(['-'])]
print(c_df_train)

                label  ...                                          sentence2
0             neutral  ...  A person is training his horse for a competition.
1       contradiction  ...      A person is at a diner, ordering an omelette.
2          entailment  ...                  A person is outdoors, on a horse.
3             neutral  ...                  They are smiling at their parents
4          entailment  ...                         There are children present
...               ...  ...                                                ...
550147  contradiction  ...           four kids won awards for 'cleanest feet'
550148        neutral  ...  four homeless children had their shoes stolen,...
550149        neutral  ...  A man in a bodysuit is competing in a surfing ...
550150  contradiction  ...  A man in a business suit is heading to a board...
550151     entailment  ...  On the beautiful blue water there is a man in ...

[549367 rows x 3 columns]


In [19]:
# check if there are missing values
c_train.select([count(when(col(c).isNull(), c)).alias(c) for c in c_train.columns]).show()
c_test.select([count(when(col(c).isNull(), c)).alias(c) for c in c_test.columns]).show()
c_dev.select([count(when(col(c).isNull(), c)).alias(c) for c in c_dev.columns]).show()

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+



In [20]:
c_train_example = []
c_test_example = []
c_dev_example = []

label2int = {"contradiction": 0, "entailment": 1, "neutral": 2, "-": 3}

for row in c_df_train.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_train_example.append(inp_example)

for row in c_df_test.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_test_example.append(inp_example)

for row in c_df_dev.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_dev_example.append(inp_example)

In [21]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),\
                               pooling_mode_mean_tokens=True,\
                               pooling_mode_cls_token=False,\
                               pooling_mode_max_tokens=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [23]:
c_train_dataloader = DataLoader(c_train_example, shuffle=True, batch_size=16) # a subset

In [24]:
# train

'''
num_epochs = 1
c_model_save_path = 'output/training_snil_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, batch_size=16, name='snil-dev')
warmup_steps = math.ceil(len(c_train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up
model.fit(train_objectives=[(c_train_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=c_model_save_path)
'''
# to save time, we use the previously trained model instead of training it everytime

'\nnum_epochs = 1\nc_model_save_path = \'output/training_snil_\'+\'bert-base-uncased\'+\'-\'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n\nevaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, batch_size=16, name=\'snil-dev\')\nwarmup_steps = math.ceil(len(c_train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up\nmodel.fit(train_objectives=[(c_train_dataloader, loss)],\n          optimizer_class=torch.optim.Adam,\n          optimizer_params={\'lr\': 2e-5},\n          epochs=num_epochs,\n          warmup_steps=warmup_steps,\n          evaluator=evaluator,\n          evaluation_steps=1000,\n          output_path=c_model_save_path)\n'

In [25]:
# cp -rf /content/output/* /content/drive/MyDrive/output # save the trained model

In [26]:
classification_model_path = '/content/drive/MyDrive/output/training_snil_bert-base-uncased-2021-12-11_15-05-32'
model = SentenceTransformer(classification_model_path)
# model = SentenceTransformer(c_model_save_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_example, batch_size=16, name='snli-test')
test_evaluator(model, output_path=classification_model_path)
# test_evaluator(model, output_path=c_model_save_path)

0.5971470976423424

# 2.3


In [27]:
#dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_example, batch_size=16, name='sts-dev')

In [28]:
'''
train = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')

from pyspark.sql.functions import ceil, floor, when
train = train.withColumn('score',when(col('score') >=0 , ceil(col('score'))).otherwise(floor(col('score'))))
# Transform the data so that it can be used to train and test
train_example = []
df_train = train.toPandas()
for row in df_train.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=int(row[4]))
  train_example.append(inp_example)
###四舍五入label，不然softmaxloss没法输出，但是报错cuda
'''

"\ntrain = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')\n\nfrom pyspark.sql.functions import ceil, floor, when\ntrain = train.withColumn('score',when(col('score') >=0 , ceil(col('score'))).otherwise(floor(col('score'))))\n# Transform the data so that it can be used to train and test\ntrain_example = []\ndf_train = train.toPandas()\nfor row in df_train.iloc:\n  row = row.values.tolist()\n  inp_example = InputExample(texts=[row[5], row[6]], label=int(row[4]))\n  train_example.append(inp_example)\n###四舍五入label，不然softmaxloss没法输出，但是报错cuda\n"

In [29]:
#dev_model_save_path = 'output/stsdev_tune'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


In [30]:
'''
from sentence_transformers import SentenceTransformer, SentencesDataset
from sentence_transformers.losses import TripletLoss
# Load pre-trained model - we are using the SNLI trained model for this example.
snli_sbert_model = SentenceTransformer(c_model_save_path)
# Set up data for fine-tuning 

finetune_data = SentencesDataset(examples=train_example, model=snli_sbert_model)
finetune_dataloader = DataLoader(finetune_data, shuffle=True, batch_size=16)

loss = losses.SoftmaxLoss(model=snli_sbert_model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)
# Fine-tune the model
snli_sbert_model.fit(train_objectives=[(finetune_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=1,
          warmup_steps=100,
          evaluator=evaluator,
          evaluation_steps=1000)
          #output_path=model_save_path)
'''

"\nfrom sentence_transformers import SentenceTransformer, SentencesDataset\nfrom sentence_transformers.losses import TripletLoss\n# Load pre-trained model - we are using the SNLI trained model for this example.\nsnli_sbert_model = SentenceTransformer(c_model_save_path)\n# Set up data for fine-tuning \n\nfinetune_data = SentencesDataset(examples=train_example, model=snli_sbert_model)\nfinetune_dataloader = DataLoader(finetune_data, shuffle=True, batch_size=16)\n\nloss = losses.SoftmaxLoss(model=snli_sbert_model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)\n# Fine-tune the model\nsnli_sbert_model.fit(train_objectives=[(finetune_dataloader, loss)],\n          optimizer_class=torch.optim.Adam,\n          optimizer_params={'lr': 2e-5},\n          epochs=1,\n          warmup_steps=100,\n          evaluator=evaluator,\n          evaluation_steps=1000)\n          #output_path=model_save_path)\n"

In [31]:
# load the classification model
classification_model_path = '/content/drive/MyDrive/output/training_snil_bert-base-uncased-2021-12-11_15-05-32'

In [32]:
# train

'''
model = SentenceTransformer(classification_model_path)
train_dataloader = DataLoader(train_example, shuffle=True, batch_size=16)
loss = losses.CosineSimilarityLoss(model=model)
num_epochs = 1
combine_model_save_path = 'output/training_combined_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name='combined-dev')
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up


model.fit(train_objectives=[(train_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=combine_model_save_path)
'''
# to save time, we use the previously trained model instead of training it everytime

'\nmodel = SentenceTransformer(classification_model_path)\ntrain_dataloader = DataLoader(train_example, shuffle=True, batch_size=16)\nloss = losses.CosineSimilarityLoss(model=model)\nnum_epochs = 1\ncombine_model_save_path = \'output/training_combined_\'+\'bert-base-uncased\'+\'-\'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\nevaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name=\'combined-dev\')\nwarmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up\n\n\nmodel.fit(train_objectives=[(train_dataloader, loss)],\n          optimizer_class=torch.optim.Adam,\n          optimizer_params={\'lr\': 2e-5},\n          epochs=num_epochs,\n          warmup_steps=warmup_steps,\n          evaluator=evaluator,\n          evaluation_steps=1000,\n          output_path=combine_model_save_path)\n'

In [34]:
combine_model_path = '/content/drive/MyDrive/output/training_combined_bert-base-uncased-2021-12-11_17-35-54'
model = SentenceTransformer(combine_model_path)
# model = SentenceTransformer(combine_model_save_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_example, batch_size=16, name='combine-test')
test_evaluator(model, output_path=combine_model_path)
# test_evaluator(model, output_path=combine_model_save_path)

0.6117927538888907

In [ ]:
# cp -rf /content/output/training_combined_bert-base-uncased-2021-12-11_17-35-54 /content/drive/MyDrive/output